# **Complex Example**

In [1]:
from dewloosh.geom import PolyData
from dewloosh.geom.space import StandardFrame, PointCloud
from dewloosh.geom.utils import centralize, center_of_points
from dewloosh.math.linalg import linspace, Vector
from dewloosh.geom.tri.triang import triangulate
from dewloosh.geom import grid
from dewloosh.geom.tri.triutils import get_points_inside_triangles
from dewloosh.geom.topo import remap_topo
from sectionproperties.pre.library.steel_sections import circular_hollow_section as CHS
from sectionproperties.pre.library.steel_sections import rectangular_hollow_section as RHS
from sectionproperties.pre.library.steel_sections import i_section
from sectionproperties.analysis.section import Section
from sectionproperties.pre.pre import Material
import numpy as np

In [2]:
steel = Material(
    name='Steel', elastic_modulus=200e3, poissons_ratio=0.3, density=7.85e-6,
    yield_strength=250, color='grey'
)

In [3]:
database = {
    'left' : {
        'geom' : CHS(d=100, t=10, n=64),
        'support' : True},
    'right' : {
        'geom' : CHS(d=100, t=10, n=64),
        'support' : True},
    'front' : {
        'geom' : RHS(d=100, b=100, t=10, r_out=0, n_r=0, material=steel),
        'dynams' : {'N' : 1e+6, 'Vx' : 0., 'Vy' : 0, 
                    'T' : 0.,  'Mx' : 0, 'My' : 0.}},
    'back' : {
        'geom' : RHS(d=100, b=100, t=10, r_out=0, n_r=0, material=steel),
        'dynams' : {'N' : 0., 'Vx' : 0., 'Vy' : 0, 
                    'T' : 5e+7,  'Mx' : 0, 'My' : 0.}},
    'top' : {
        'geom' : i_section(d=170, b=110, t_f=7.8, t_w=5.8, r=8.9, n_r=16, material=steel),
        'dynams' : {'N' : 0., 'Vx' : 1e+4, 'Vy' : 0, 
                    'T' : 0,  'Mx' : 0, 'My' : 0.}}
}

In [4]:
Lx, Ly, Lz = 220, 220, 220
points_per_edge = 25
mesh_size = Lx / (points_per_edge-1)

In [5]:
points = []
fixity = []
loads = []
nTotalPoints = 0  # node counter

# corners
corner_coords = [
    [-Lx/2, -Ly/2, -Lz/2],
    [Lx/2, -Ly/2, -Lz/2],
    [Lx/2, Ly/2, -Lz/2],
    [-Lx/2, Ly/2, -Lz/2],
    [-Lx/2, -Ly/2, Lz/2],
    [Lx/2, -Ly/2, Lz/2],
    [Lx/2, Ly/2, Lz/2],
    [-Lx/2, Ly/2, Lz/2]
]
corner_coords = np.array(corner_coords)
sig_corner = np.zeros((corner_coords.shape[0], 3)).astype(float)
fixity_corner = np.zeros((corner_coords.shape[0], 3)).astype(bool)
points.append(corner_coords)
fixity.append(fixity_corner)
loads.append(sig_corner)
nTotalPoints += len(corner_coords)

# populate edges
nodes_of_edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],
    [4, 5], [5, 6], [6, 7], [7, 4],
    [0, 4], [1, 5], [2, 6], [3, 7]
    ]
edge_coords = []
N = points_per_edge + 2
for nodes in nodes_of_edges:
    p0 = corner_coords[nodes[0]]
    p1 = corner_coords[nodes[1]]
    edge_coords.append(linspace(p0, p1, N)[1:-1])
edge_coords = np.vstack(edge_coords)
sig_edge = np.zeros((edge_coords.shape[0], 3)).astype(float)
fixity_edge = np.zeros((edge_coords.shape[0], 3)).astype(bool)
points.append(edge_coords)
fixity.append(fixity_edge)
loads.append(sig_edge)
nTotalPoints += len(edge_coords)

# faces
corners_of_faces = {
    'front' : [1, 2, 6, 5], 
    'back' : [0, 3, 7, 4], 
    'left' : [2, 3, 7, 6],  
    'right' : [0, 1, 5, 4],
    'bottom' : [0, 1, 2, 3], 
    'top' : [4, 5, 6, 7],  
}
edges_of_faces = {
    'front' : [1, 5, 9, 10], 
    'back' : [3, 7, 8, 11], 
    'right' : [0, 9, 4, 8],  
    'left' : [2, 6, 10, 11],
    'bottom' : [0, 1, 2, 3], 
    'top' : [4, 5, 6, 7],  
}

# center of face
def cof(id) : return center_of_points(corner_coords[corners_of_faces[id]])

# face frames
frames = {}
GlobalFrame = StandardFrame(dim=3)
frames['front'] = GlobalFrame.fork().move(cof('front'))
rot90z = 'Body', [0, 0, np.pi/2], 'XYZ'
frames['left'] = frames['front'].fork(*rot90z).move(cof('left') - cof('front'))
frames['back'] = frames['left'].fork(*rot90z).move(cof('back') - cof('left'))
frames['right'] = frames['back'].fork(*rot90z).move(cof('right') - cof('back'))
rot_front_top = 'Body', [0, -np.pi/2, 0], 'XYZ'
frames['top'] = frames['front'].fork(*rot_front_top).move(cof('top') - cof('front'))
rot180y = 'Body', [0, np.pi, 0], 'XYZ'
frames['bottom'] = frames['top'].fork(*rot180y).move(cof('bottom') - cof('top'))
#frames = {}

In [6]:
coords_grid, topo_grid = grid(size=(Lx*0.99, Ly*0.99), shape=(N, N), 
                              eshape='Q4', centralize=True)
Grid = PolyData(coords=coords_grid, topo=topo_grid, frame=GlobalFrame)
grid_centers = Grid.centers()[:, :2]
fixity_grid = np.zeros((grid_centers.shape[0], 3)).astype(bool)
sig_grid = np.zeros((grid_centers.shape[0], 3)).astype(float)

In [7]:
for face in frames:
    
    f_frame = frames[face]    
    # collect points on corners and edges and their global indices
    # these indices are used later to remap the default topology
    # resulting from per-face triangulations
    f_coords_base = []
    f_inds_base = []
    _corner_inds = []
    for corner in corners_of_faces[face]:
        f_coords_base.append(corner_coords[corner])
        _corner_inds.append(corner)
    f_inds_base.append(np.array(_corner_inds, dtype=int))
    for edge in edges_of_faces[face]:
        inds = np.arange(points_per_edge) + edge * points_per_edge
        f_coords_base.append(edge_coords[inds])
        f_inds_base.append(inds + 8)
    f_coords_base = np.vstack(f_coords_base)
    sig_coords_base = np.zeros((f_coords_base.shape[0], 3)).astype(float)
    fixity_coords_base = np.zeros((f_coords_base.shape[0], 3)).astype(bool)
        
    # transform the coords (base coordinates) so far to face frame    
    f_coords_base = PointCloud(f_coords_base, frame=GlobalFrame).show(f_frame)    
    #f_coords_base = yz_to_xy(f_coords_base)[:, :2]
    
    # global indices and number of corner and edge points
    f_inds_base = np.concatenate(f_inds_base)
    nBasePoints = len(f_inds_base)
    
    # build face    
    if face in database:
        # 1) create the mesh of the section
        # 2) rule out points of the base grid that the section covers
        # 3) add corner and edge nodes and do a triangulation 
        f_section = database[face]['geom']
        f_section.create_mesh(mesh_sizes=[20.])
        f_coords = centralize(np.array(f_section.mesh['vertices']))
        
        # boundary conditions
        sig_coords = np.zeros((f_coords.shape[0], 3)).astype(float)
        fixity_coords = np.zeros((f_coords.shape[0], 3)).astype(bool)
        if 'dynams' in database[face]:
            dyn = database[face]['dynams']
            _section = Section(f_section)
            _section.calculate_geometric_properties()
            _section.calculate_warping_properties()
            stress_post = _section.calculate_stress(
                N=dyn['N'], Vy=dyn['Vy'], Vx=dyn['Vx'], 
                Mzz=dyn['T'], Mxx=dyn['Mx'], Myy=dyn['My']
                )
            stresses = stress_post.get_stress()
            sig_coords[:, 0] = stresses[0]['sig_zz']
            sig_coords[:, 1] = stresses[0]['sig_zx']
            sig_coords[:, 2] = stresses[0]['sig_zy']
            sig_coords = Vector(sig_coords, frame=f_frame).show(GlobalFrame)
        elif 'support' in database[face]:
            fixity_coords[:, :] = True
        else:
            raise NotImplementedError
    
        n_section_nodes = f_coords.shape[0]
        f_topo = np.array(f_section.mesh['triangles'].tolist())[:, :3]
        f_inds = get_points_inside_triangles(f_coords, f_topo, grid_centers).astype(bool)      
        f_coords = np.vstack([f_coords_base[:, 1:], f_coords, grid_centers[~f_inds]])     
        f_coords, f_topo, _ = triangulate(points=f_coords)
        f_sig = np.vstack([sig_coords_base, sig_coords, sig_grid[~f_inds]])
        f_fixity = np.vstack([fixity_coords_base, fixity_coords, fixity_grid[~f_inds]]) 
    else:
        f_coords = np.vstack([f_coords_base[:, 1:], grid_centers])
        f_coords, f_topo, _ = triangulate(points=f_coords)
        f_sig = np.zeros((f_coords.shape[0], 3)).astype(float)
        f_fixity = np.zeros((f_coords.shape[0], 3)).astype(bool)
                
    # faces share some points, hence they must be consistent
    # in node numbering --> remap topology to match indices
    # of corner and edge nodes
    f_inds = np.zeros(len(f_coords), dtype=int)
    nNewPoints = len(f_coords) - nBasePoints
    f_inds[:nBasePoints] = f_inds_base
    f_inds[nBasePoints:] = np.arange(nNewPoints) + nTotalPoints
    nTotalPoints += nNewPoints
    f_topo = remap_topo(f_topo, f_inds)
    
    # transform to global and append new data to total collection
    f_coords_new = np.zeros((nNewPoints, 3))
    f_coords_new[:, 1:] = f_coords[nBasePoints:]
    f_coords = PointCloud(f_coords_new, frame=f_frame).show(GlobalFrame)
    points.append(f_coords)
    fixity.append(f_fixity[nBasePoints:])
    loads.append(f_sig[nBasePoints:])
    
    # add topology to face to total collection
    if face not in database:
        database[face] = {}
    database[face]['topo'] = f_topo

In [8]:
cubepoints = np.vstack(points)
nCubePoints = len(cubepoints)
cube = PolyData(coords=cubepoints, frame=GlobalFrame)
for face in frames:
    cube[face] = PolyData(topo=database[face]['topo'])

In [9]:
cube.plot()